In [86]:
import os
from six import moves
import ssl

import tflearn
from tflearn.data_utils import *

In [99]:
import pandas as pd
df = pd.read_csv('villes_france.csv',low_memory=False)
df.head()

,Département,Slug,Nom,Nom simple,Nom reel,Nom soundex,Nom metaphone,Code Postal,Numéro de commune,Code INSEE,...,Densité en 2010,Superficie,Longitude (degré),Latitude (degré),Longitude (GRD),Latitude (GRD),Longitude (DMS),Latitude (DMS),Altitude minimale,Altitude maximale
0,1,ozan,OZAN,ozan,Ozan,O250,OSN,1190,284,1284,...,93,6.60,4.91667,46.3833,2866.0,51546.0,45456.0,462330.0,170.0,205.0
1,1,cormoranche-sur-saone,CORMORANCHE-SUR-SAONE,cormoranche sur saone,Cormoranche-sur-Saône,C65652625,KRMRNXSRSN,1290,123,1123,...,107,9.85,4.83333,46.2333,2772.0,51379.0,44953.0,461427.0,168.0,211.0
2,1,plagne-01,PLAGNE,plagne,Plagne,P425,PLKN,1130,298,1298,...,20,6.20,5.73333,46.1833,3769.0,51324.0,54342.0,461131.0,560.0,922.0
3,1,tossiat,TOSSIAT,tossiat,Tossiat,T230,TST,1250,422,1422,...,138,10.17,5.31667,46.1333,3309.0,51268.0,51854.0,460828.0,244.0,501.0
4,1,pouillat,POUILLAT,pouillat,Pouillat,P430,PLT,1250,309,1309,...,14,6.23,5.43333,46.3333,3435.0,51475.0,52542.0,461938.0,333.0,770.0


In [100]:
import sqlite3
conn = sqlite3.connect('villes_france.sql')
pandas.read_sql('villes_france',conn)

DatabaseError: Execution failed on sql 'villes_france': near "villes_france": syntax error

In [88]:
villes = list(df['Nom simple'])
all_villes = ''
for ville in villes:
    all_villes += ville +'\n'

In [89]:
villes

['ozan',
 'cormoranche sur saone',
 'plagne',
 'tossiat',
 'pouillat',
 'torcieu',
 'replonges',
 'corcelles',
 'peron',
 'relevant',
 'chaveyriat',
 'vaux en bugey',
 'maillat',
 'faramans',
 'beon',
 'saint bernard',
 'rossillon',
 'pont d ain',
 'nantua',
 'chavannes sur reyssouze',
 'neuville les dames',
 'flaxieu',
 'hotonnes',
 'saint sorlin en bugey',
 'songieu',
 'virieu le petit',
 'saint denis en bugey',
 'charnoz sur ain',
 'chazey sur ain',
 'marchamp',
 'culoz',
 'mantenay montlin',
 'marboz',
 'foissiat',
 'treffort cuisiat',
 'izieu',
 'saint etienne du bois',
 'hauteville lompnes',
 'saint trivier sur moignans',
 'peyriat',
 'evosges',
 'poncin',
 'crans',
 'montreal la cluse',
 'cleyzieu',
 'lompnieu',
 'villereversure',
 'saint martin du mont',
 'saint genis pouilly',
 'bolozon',
 'confrancon',
 'lochieu',
 'chanoz chatenay',
 'villebois',
 'ceignes',
 'saint didier sur chalaronne',
 'revonnas',
 'bourg saint christophe',
 'condeissiat',
 'pirajoux',
 'chalamont',
 'l

In [90]:
context = ssl._create_unverified_context()
moves.urllib.request.urlretrieve("https://raw.githubusercontent.com/tflearn/tflearn.github.io/master/resources/US_Cities.txt", path)

('US_Cities.txt', <http.client.HTTPMessage at 0x4630a9ef0>)

In [91]:
maxlen = 30

#cities = open(path, "r").read()#.decode('utf-8')
cities = all_villes
X, Y, char_idx = \
    string_to_semi_redundant_sequences(cities, seq_maxlen=maxlen, redun_step=3)

Vectorizing text...
Text total length: 467,173
Distinct chars   : 30
Total sequences  : 155,715


In [92]:
g = tflearn.input_data(shape=[None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model_us_cities')

In [93]:
for i in range(40):
    seed = random_sequence_from_string(cities, maxlen)
    m.fit(X, Y, validation_set=0.1, batch_size=128,
          n_epoch=1, run_id='us_cities')
    print("-- TESTING...")
    print("-- Test with temperature of 1.2 --")
    print(m.generate(30, temperature=1.2, seq_seed=seed).encode('utf-8'))
    print("-- Test with temperature of 1.0 --")
    print(m.generate(30, temperature=1.0, seq_seed=seed).encode('utf-8'))
    print("-- Test with temperature of 0.5 --")
    print(m.generate(30, temperature=0.5, seq_seed=seed).encode('utf-8'))

IndexError: list index out of range

In [ ]:
random_sequence_from_string(cities, maxlen)